[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_28/VL28_BERT_Embeddings.ipynb)

# VL28 Notebook 2: BERT Embeddings
## Kontextabhängige Wort-Vektoren

In diesem Notebook lernen wir:
- Wie BERT **denselben Wort verschiedene Vektoren** gibt (je nach Kontext)
- Das "Bank"-Beispiel: Geldinstitut vs. Sitzmöbel
- Wie man BERT für Sentiment-Analyse nutzt
- **Kein API-Key nötig** - alles läuft lokal mit Hugging Face!

In [ ]:
# Installation (nur einmal ausführen)
# !pip install transformers torch numpy scikit-learn

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

## 1. BERT Modell laden

Wir nutzen ein deutsches BERT-Modell von Hugging Face (kostenlos, kein API-Key).

In [ ]:
# Deutsches BERT-Modell laden
model_name = "bert-base-german-cased"
print(f"Lade Modell: {model_name}")

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

print("✓ Modell geladen!")

## 2. Das "Bank"-Beispiel: Zwei Kontexte, zwei Vektoren

**Word2Vec Problem:** "Bank" hat EINEN Vektor, egal ob Geldinstitut oder Sitzmöbel.

**BERT Lösung:** "Bank" bekommt unterschiedliche Vektoren je nach Kontext!

In [ ]:
# Zwei Sätze mit unterschiedlicher Bedeutung von "Bank"
satz1 = "Ich gehe zur Bank und hole Geld."
satz2 = "Ich sitze auf der Bank im Park."

def get_word_embedding(sentence, word, tokenizer, model):
    """Extrahiert den Embedding-Vektor für ein Wort in einem Satz."""
    # Tokenisierung
    inputs = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.tokenize(sentence)
    
    # BERT durchlaufen (ohne Gradienten)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Letzter Hidden State: [batch_size, seq_len, hidden_size]
    last_hidden = outputs.last_hidden_state[0]  # [seq_len, 768]
    
    # Finde Position des Wortes
    word_lower = word.lower()
    for i, token in enumerate(tokens):
        if word_lower in token.lower():
            # +1 wegen [CLS] Token am Anfang
            return last_hidden[i+1].numpy()
    
    return None

# Embeddings für "Bank" in beiden Kontexten
vec1 = get_word_embedding(satz1, "Bank", tokenizer, model)
vec2 = get_word_embedding(satz2, "Bank", tokenizer, model)

# Ähnlichkeit berechnen
similarity = cosine_similarity([vec1], [vec2])[0][0]

print("Satz 1:", satz1)
print("Satz 2:", satz2)
print(f"\nCosine-Ähnlichkeit der 'Bank'-Vektoren: {similarity:.4f}")
print(f"\nDie Vektoren sind unterschiedlich!")
print(f"→ BERT erkennt den Kontext und gibt 'Bank' verschiedene Bedeutungen.")

## 3. Visualisierung: Vergleich mit Word2Vec

**Vergleich:**
- Word2Vec: "Bank" hätte Ähnlichkeit = 1.0 (identischer Vektor)
- BERT: "Bank" hat Ähnlichkeit < 1.0 (unterschiedliche Vektoren)

In [ ]:
# Weitere Beispiele mit mehrdeutigen Wörtern
beispiele = [
    ("Das Schloss ist sehr alt.", "Ich öffne das Schloss mit einem Schlüssel.", "Schloss"),
    ("Der Ball ist rund.", "Wir gehen zum Ball und tanzen.", "Ball"),
]

for satz_a, satz_b, wort in beispiele:
    vec_a = get_word_embedding(satz_a, wort, tokenizer, model)
    vec_b = get_word_embedding(satz_b, wort, tokenizer, model)
    
    if vec_a is not None and vec_b is not None:
        sim = cosine_similarity([vec_a], [vec_b])[0][0]
        print(f"\n{wort}:")
        print(f"  A: {satz_a}")
        print(f"  B: {satz_b}")
        print(f"  Ähnlichkeit: {sim:.4f}")

## 4. Sentiment-Analyse mit BERT (Bonus)

BERT kann auch für Klassifikation verwendet werden. Hier ein einfaches Sentiment-Beispiel.

In [ ]:
from transformers import pipeline
import os

# Erzwinge PyTorch (kein TensorFlow)
os.environ['TRANSFORMERS_NO_TF'] = '1'

# Englisches Sentiment-Modell (klein, schnell, PyTorch-only)
print("Lade Sentiment-Modell (Englisch)...")
sentiment = pipeline("sentiment-analysis", 
                     model="distilbert-base-uncased-finetuned-sst-2-english",
                     framework="pt")  # Explizit PyTorch verwenden

# Test-Sätze (Englisch)
sätze = [
    "This product is fantastic!",
    "I am very disappointed with the quality.",
    "It is okay, nothing special.",
]

print("✓ Modell geladen!\n")
print("Sentiment-Analyse:\n")
for satz in sätze:
    result = sentiment(satz)[0]
    label_de = "POSITIV" if result['label'] == "POSITIVE" else "NEGATIV"
    print(f"Satz: {satz}")
    print(f"  Sentiment: {label_de}")
    print(f"  Konfidenz: {result['score']:.2%}")
    print()

## 🎯 Takeaway

**BERT Vorteile gegenüber Word2Vec:**
- ✅ **Kontextabhängige Embeddings**: "Bank" bekommt verschiedene Vektoren je nach Bedeutung
- ✅ **Bidirektionaler Kontext**: BERT liest den ganzen Satz (links UND rechts)
- ✅ **Pre-trained auf riesigen Korpora**: Starkes Sprachwissen "out of the box"
- ✅ **Vielseitig**: Klassifikation, Q&A, Named Entity Recognition, etc.

**Wie BERT das macht:**
- Transformer-Architektur mit **Self-Attention**
- Jedes Wort "schaut" auf alle anderen Wörter im Satz
- Der finale Vektor enthält Information aus dem gesamten Kontext